In [1]:
cd /home/CAMPUS/hdasari/Hypernetworks_stevens

/home/CAMPUS/hdasari/Hypernetworks_stevens


In [2]:
import torch
from torch.utils.data import DataLoader
from src.models.Unet_model import UNet, UNetTinyTwoBlock



device = torch.device("cpu")
print(f"Using device: {device}")


unet_path = "/home/CAMPUS/sgangadh1/projects/rl-batt-seg-snapshot-jan-2024/src/outputs/rerun-battery-01/unet_model_checkpoint_finetuned.pt"
global_model_path = "/home/CAMPUS/hdasari/HyperNetworks/model_checkpoints_AAI_paper/unet_tiny_two_block_model_checkpoint_199_final.pt"

expert_model_unet = UNet(n_channels=1, n_classes=3)
expert_model_unet = expert_model_unet.to(device)

global_model = UNetTinyTwoBlock(n_channels=1, n_classes=3)
global_model = global_model.to(device)


expert_checkpoint = torch.load(unet_path)
global_checkpoint = torch.load(global_model_path)

expert_model_unet.load_state_dict(expert_checkpoint['model_state_dict'])
expert_model_unet.eval()

global_model.load_state_dict(global_checkpoint['model_state_dict'])
global_model.eval()

Using device: cpu


/tmp/ipykernel_1378390/3870493577.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  expert_checkpoint = torch.load(unet_path)
/tmp/ipykernel_1378390/3870493577.py:22: Fut

UNetTinyTwoBlock(
  (softconv): Conv2d(1, 4, kernel_size=(1, 1), stride=(1, 1))
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): InstanceNorm2d(8, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): ReLU(inplace=True)
      (3): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): InstanceNorm2d(8, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): InstanceNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
          (2): ReLU(inplace=True)
          (3): Conv2d(16, 16, kernel_size=(3, 3), s

In [3]:
import torch
from torch.utils.data import Dataset
import os
from PIL import Image
import numpy as np
import glob
from torchvision import transforms
from pathlib import Path
from src.utils.hyp_input import hyp_input
from src.utils.get_boundary_pixels import get_boundary_pixels
from src.utils.extract_patch import extract_patch


class Battery_unet_hyp_data(Dataset):
    def __init__(self, image_dir,label_dir,expert_model,small_model, device, mask_function=hyp_input, get_boundaries=get_boundary_pixels, get_patch=extract_patch, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.mask_function = mask_function
        self.get_boundaries = get_boundaries
        self.get_patch = get_patch
        self.transform = transform
        self.expert_model = expert_model
        self.small_model = small_model
        self.device = device
        
        self.image_files = sorted(Path(image_dir).glob('*.png'))
        self.label_files = sorted(Path(label_dir).glob('*.png'))
        assert len(self.image_files) == len(self.label_files), "Number of image and label files must be the same!"

        
    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        label_path = self.label_files[idx]
        label = Image.open(label_path)

        mask_transform = transforms.Compose([
            transforms.Lambda(lambda x: x.convert('L') if x.mode != 'L' else x),
            transforms.Lambda(lambda x: np.array(x, dtype=np.float32) / 255.0),
            transforms.Lambda(lambda x: np.where((x > 0) & (x < 1.0), 2.0, x)),
            transforms.Lambda(lambda x: torch.as_tensor(x.copy()).long()),
        ])
        label_tensor = mask_transform(label) 

        # Load and transform images
        image = Image.open(image_path).convert('L')
        img_ndarray = np.asarray(image)
        img_ndarray = img_ndarray[np.newaxis, ...]  # Add channel dimension [1, H, W]
        image_tensor = torch.as_tensor(img_ndarray / 255.0).float().contiguous()
        
        with torch.no_grad():
            input_img = image_tensor.unsqueeze(0).to(self.device)  # Add batch dimension
            expert_label_tensor = self.expert_model(input_img)
            small_label_tensor = self.small_model(input_img)
        expert_label_tensor = expert_label_tensor.squeeze(0).cpu().type(torch.long)
        small_label_tensor = small_label_tensor.squeeze(0).cpu().type(torch.long)

        # print(f"expert_label_tensor shape: {expert_label_tensor.shape}")
        # print(f"small_label_tensor shape: {small_label_tensor.shape}")

        

        _, H, W = image_tensor.shape

        # Get key pixels and masked image
        key_pixels, expert_patches, global_patches = self.mask_function(expert_label_tensor, small_label_tensor)
        



        # print(f"masked_img shape: {expert_patches.shape}")
        # print(f"global_patches shape: {global_patches.shape}")

        all_patches = []
        all_labels = []

        mismatch = 0

        for x, y in key_pixels:
            boundary_pixels = self.get_boundaries(x, y)

            patches = []
            labels = []

            for bx, by in boundary_pixels:
                if bx < 0 or by < 0 or bx >= H or by >= W:
                    patches.append(torch.zeros((1, 9, 9), dtype=torch.long))
                    labels.append(255)
                    mismatch += 1
                else:
                    patches.append(self.get_patch(image_tensor, bx, by))
                    labels.append(label_tensor[bx, by])  # Get label ID

            all_patches.append(torch.stack(patches))  # Shape: (max_boundaries, C, H, W)
            all_labels.append(torch.tensor(labels, dtype=torch.long))

        # Convert lists to tensors
        all_patches = torch.stack(all_patches)  # Shape: (num_key_pixels, max_boundaries, C, H, W)
        all_labels = torch.stack(all_labels)  # Shape: (num_key_pixels, max_boundaries)

        return all_patches, global_patches, expert_patches, all_labels, mismatch


In [5]:


image_dir = "/home/CAMPUS/sgangadh1/projects/rl-batt-seg-snapshot-jan-2024/data/battery_2/train_images"
label_dir = "/home/CAMPUS/sgangadh1/projects/rl-batt-seg-snapshot-jan-2024/data/battery_2/train_label"

# Load dataset
train_dataset = Battery_unet_hyp_data(image_dir, label_dir, expert_model_unet, global_model,device)


# Iterate through the DataLoader and inspect the output
def display_first_batch_shape(dataset, batch_size=1):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    for batch in dataloader:
        all_patches, global_patches, expert_patches, all_labels, mismatch = batch
        print(f"all_patches length: {len(all_patches)}") 
        print(f"all_patches shape: {(all_patches.shape)}") # Expected: (batch_size, K, 40, C, H, W)
        print(f"global shape: {global_patches.shape}")  # Expected: (batch_size, k,concatednated_patch[9*9+8])
        print(f"expert shape: {expert_patches.shape}")  # Expected: (batch_size, K,2)
        print(f"all_labels shape: {all_labels.shape}")  # Expected: (batch_size, K, 40)
        print(f"Mismatches: {mismatch}")
        break
       
        
        

       
display_first_batch_shape(train_dataset, batch_size=16)


all_patches length: 16
all_patches shape: torch.Size([16, 3, 40, 1, 9, 9])
global shape: torch.Size([16, 3, 371])
expert shape: torch.Size([16, 3, 251])
all_labels shape: torch.Size([16, 3, 40])
Mismatches: tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 17])


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter

class HyperNetwork(nn.Module):
    def __init__(self, f_size=3, z_dim=64, out_size=16, in_size=16):
        super(HyperNetwork, self).__init__()
        self.z_dim = z_dim
        self.f_size = f_size
        self.out_size = out_size
        self.in_size = in_size

        # Define parameters and move them to GPU
        self.w1 = Parameter(torch.fmod(torch.randn((16, self.out_size * self.in_size * self.f_size * self.f_size)), 2))
        self.b1 = Parameter(torch.fmod(torch.randn((self.out_size * self.in_size * self.f_size * self.f_size)), 2))

        self.w_global_img = Parameter(torch.fmod(torch.randn((371, 64)), 2))
        self.b_global_img = Parameter(torch.fmod(torch.randn((64)), 2))

        self.w_expert_img = Parameter(torch.fmod(torch.randn((251, 64)), 2))
        self.b_expert_img = Parameter(torch.fmod(torch.randn((64)), 2))

        self.dense_w = Parameter(torch.fmod(torch.randn((144, 16)), 2))
        self.dense_b = Parameter(torch.fmod(torch.randn((16)), 2))
        
        self.w2 = Parameter(torch.fmod(torch.randn((self.z_dim, self.in_size)), 2))
        self.b2 = Parameter(torch.fmod(torch.randn(self.in_size), 2))

    def forward(self, z, expert_patch, global_patch):

        h_z = torch.matmul(z, self.w2) + self.b2
        h_z = torch.tanh(h_z)
        
        # y_global_img = torch.squeeze(global_patch, dim=0).float()
        y_global_prime = torch.matmul(global_patch, self.w_global_img) + self.b_global_img
        y_global_prime = torch.tanh(y_global_prime)

        
        # y_expert_img = torch.squeeze(expert_patch, dim=0).float()
        y_expert_prime = torch.matmul(expert_patch, self.w_expert_img) + self.b_expert_img
        h_img_trans = torch.cat([y_global_prime, y_expert_prime, h_z], dim = 0)
        h_prime = torch.matmul(h_img_trans, self.dense_w) + self.dense_b  
        h_prime = torch.tanh(h_prime)      
        h_final = torch.matmul(h_prime, self.w1) + self.b1
        h_final = torch.tanh(h_final)
        kernel = h_final.view(self.out_size, self.in_size, self.f_size, self.f_size)

        return kernel

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class IdentityLayer(nn.Module):
    def forward(self, x):
        return x

class ResNetBlock(nn.Module):
    def __init__(self, in_size=16, out_size=16, downsample=False):
        super(ResNetBlock, self).__init__()
        self.out_size = out_size
        self.in_size = in_size
        if downsample:
            self.stride1 = 2
            self.reslayer = nn.Conv2d(in_channels=self.in_size, out_channels=self.out_size, stride=1, kernel_size=3, padding=1) # some mistake. need to fix it
        else:
            self.stride1 = 1
            self.reslayer = IdentityLayer()

        self.bn1 = nn.BatchNorm2d(out_size)
        self.bn2 = nn.BatchNorm2d(out_size)

    def forward(self, x, conv1_w, conv2_w):
        x = x
        residual = self.reslayer(x)
        # print(f"Residual shape: {residual.shape}")

        out = F.relu(self.bn1(F.conv2d(x, conv1_w, stride=1, padding=1)), inplace=True)
        # print(f"Conv1 shape: {out.shape}")
        out = self.bn2(F.conv2d(out, conv2_w, padding=1))

        out += residual
        # print(f"Out shape: {out.shape}")

        out = F.relu(out)

        return out


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter



class Embedding(nn.Module):

    def __init__(self, z_num, z_dim):
        super(Embedding, self).__init__()

        self.z_list = nn.ParameterList()
        self.z_num = z_num
        self.z_dim = z_dim

        h,k = self.z_num

        for i in range(h):
            for j in range(k):
                self.z_list.append(Parameter(torch.fmod(torch.randn(self.z_dim).cuda(), 2)))

    def forward(self, hyper_net, expert_patch, global_patch):
        ww = []
        h, k = self.z_num
        for i in range(h):
            w = []
            for j in range(k):
                w.append(hyper_net(self.z_list[i*k + j],expert_patch, global_patch))
            ww.append(torch.cat(w, dim=1))
        return torch.cat(ww, dim=0)


class PrimaryNetwork(nn.Module):

    def __init__(self, z_dim=64):
        super(PrimaryNetwork, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)

        self.z_dim = z_dim
        self.hope = HyperNetwork(z_dim=self.z_dim)

        self.zs_size = [[1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1],
                        [2, 1], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2],
                        [4, 2], [4, 4], [4, 4], [4, 4], [4, 4], [4, 4], [4, 4], [4, 4], [4, 4], [4, 4], [4, 4], [4, 4]]

        self.filter_size = [[16,16], [16,16], [16,16], [16,16], [16,16], [16,16], [16,32], [32,32], [32,32], [32,32],
                            [32,32], [32,32], [32,64], [64,64], [64,64], [64,64], [64,64], [64,64]]

        self.res_net = nn.ModuleList()

        for i in range(18):
            down_sample = False
            if i > 5 and i % 6 == 0:
                down_sample = True
            self.res_net.append(ResNetBlock(self.filter_size[i][0], self.filter_size[i][1], downsample=down_sample))

        self.zs = nn.ModuleList()

        for i in range(36):
            self.zs.append(Embedding(self.zs_size[i], self.z_dim))

        self.global_avg = nn.AvgPool2d(9)
        self.final = nn.Linear(64,3) # change for number of classes

    def forward(self, x,expert_patch,global_patch):

        x = F.relu(self.bn1(self.conv1(x)))
        for i in range(18):
            w1 = self.zs[2*i](self.hope, expert_patch, global_patch)
            w2 = self.zs[2*i+1](self.hope, expert_patch, global_patch)            
            x = self.res_net[i](x, w1, w2)
            

        x = self.global_avg(x)
        x= x.squeeze(-1).squeeze(-1)
        x = self.final(x)

        return x

In [9]:
import torch


# Assuming CUDA is available
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

# Instantiate the model
model = PrimaryNetwork(z_dim=64).to(device)

# Dummy input: Batch of 4 RGB images of size 72x72 (adjust as per your needs)
x = torch.randn(40,1,9,9).to(device)

# Dummy expert_patch and global_patch
# Adjust dimensions as required by your HyperNetwork
expert_patch = torch.randn(251).to(device)
global_patch = torch.randn(371).to(device)

# Forward pass
with torch.no_grad():
    output = model(x, expert_patch, global_patch)

print("Output shape:", output.shape)  # Should be [4, 3] if final layer has 20 output classes


Output shape: torch.Size([40, 3])


In [15]:
from torchinfo import summary
model = PrimaryNetwork(z_dim=64).to("cuda:3" if torch.cuda.is_available() else "cpu")
summary(model, input_data=(x, expert_patch, global_patch))

Layer (type:depth-idx)                   Output Shape              Param #
PrimaryNetwork                           [40, 3]                   --
├─Conv2d: 1-1                            [40, 16, 9, 9]            160
├─BatchNorm2d: 1-2                       [40, 16, 9, 9]            32
├─ModuleList: 1-281                      --                        (recursive)
│    └─Embedding: 2-1                    [16, 16, 3, 3]            64
├─HyperNetwork: 1-4                      [16, 16, 3, 3]            82,464
├─ModuleList: 1-281                      --                        (recursive)
│    └─Embedding: 2-2                    [16, 16, 3, 3]            64
├─HyperNetwork: 1-6                      [16, 16, 3, 3]            (recursive)
├─ModuleList: 1-298                      --                        (recursive)
│    └─ResNetBlock: 2-3                  [40, 16, 9, 9]            --
│    │    └─IdentityLayer: 3-1           [40, 16, 9, 9]            --
│    │    └─BatchNorm2d: 3-2             [40

In [3]:
import torch
from torch.utils.data import DataLoader
from src.models.Unet_model import UNet, UNetTinyTwoBlock
from src.data_loaders.Battery_unet_hyp_dataloader import Battery_unet_hyp_data
from src.models.primaryNet import PrimaryNetwork
import os
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import numpy as np
from torchmetrics.functional import jaccard_index




device_1 = torch.device("cpu")
print(f"Using device for unet and unet_tiny models : {device_1}")
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

unet_path = "/home/CAMPUS/sgangadh1/projects/rl-batt-seg-snapshot-jan-2024/src/outputs/rerun-battery-01/unet_model_checkpoint_finetuned.pt"
global_model_path = "/home/CAMPUS/hdasari/HyperNetworks/model_checkpoints_AAI_paper/unet_tiny_two_block_model_checkpoint_199_final.pt"

expert_model_unet = UNet(n_channels=1, n_classes=3)
expert_model_unet = expert_model_unet.to(device_1)

global_model = UNetTinyTwoBlock(n_channels=1, n_classes=3)
global_model = global_model.to(device_1)


expert_checkpoint = torch.load(unet_path)
global_checkpoint = torch.load(global_model_path)

expert_model_unet.load_state_dict(expert_checkpoint['model_state_dict'])
expert_model_unet.eval()

global_model.load_state_dict(global_checkpoint['model_state_dict'])
global_model.eval()


image_dir = "/home/CAMPUS/sgangadh1/projects/rl-batt-seg-snapshot-jan-2024/data/battery_2/train_images"
label_dir = "/home/CAMPUS/sgangadh1/projects/rl-batt-seg-snapshot-jan-2024/data/battery_2/train_label"


# Load dataset
train_dataset = Battery_unet_hyp_data(image_dir, label_dir, expert_model_unet, global_model,device_1)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)


model = PrimaryNetwork()
model = model.to(device)

criterion = nn.CrossEntropyLoss(ignore_index=255)  
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 100
start_epoch = 0
end_epoch = 100
num_batches = 1

checkpoint_path = "/home/CAMPUS/hdasari/HyperNetworks/unet_hyp_apr9/checkpoint_epoch_0.pth"  

# Load checkpoint if available
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']  
    print(f"Resuming training from epoch {start_epoch}")
else:
    start_epoch = 0  # Start from scratch if no checkpoint found
    print("No checkpoint found. Starting from epoch 0.")

# Initialize lists to store true and predicted labels
# Training loop
total_true_labels = []
total_preds = []
loss_per_epoch = []
losses = []

# Training loop
for epoch in range(1):
    model.train()
    loss_epoch = 0
    loss_per_batch = []
    all_preds = []
    all_true_labels = []
    num_left_count = 0  # Reset for each epoch
    data_len = 0  # Track the number of valid batches

    for all_patches, global_patches, expert_patches, all_labels,mismatch in train_loader:

        for batch in range(len(all_patches)):
            batch_patches = all_patches[batch].to(device)
            batch_global_patches = global_patches[batch].to(device)
            batch_expert_patches = expert_patches[batch].to(device)
            batch_labels = all_labels[batch].to(device)
            mismatch_count = mismatch[batch]

            
            batch_loss = 0
            count = 0  # Count of skipped pixels

            for patch in range(len(batch_patches)):

                optimizer.zero_grad()
                output = model(batch_patches[patch],batch_expert_patches[patch],batch_global_patches[patch])
                loss = criterion(output, batch_labels[patch])

                batch_loss += loss

                # Get predictions
                preds = output.argmax(dim=-1)
                preds_np = preds.cpu().numpy().flatten()
                true_np = batch_labels[patch].cpu().numpy().flatten()

                # Only keep entries where the ground truth is not 255
                valid_mask = true_np != 255

                # Filter predictions and ground truths
                filtered_preds = preds_np[valid_mask]
                filtered_true = true_np[valid_mask]

                # Append the filtered results
                all_preds.extend(filtered_preds)
                total_preds.extend(filtered_preds)
                all_true_labels.extend(filtered_true)
                total_true_labels.extend(filtered_true)

                loss.backward()
                optimizer.step()

            # Normalize batch_loss by valid pixels
            if len(batch_expert_patches) - count > 0:
                batch_loss /= (len(batch_expert_patches) - count)
                loss_per_batch.append(batch_loss.item())  # Storing batch loss only if valid
                loss_epoch += batch_loss.item()
                data_len += 1  # Increment count of valid batches

    # Compute metrics
    epoch_loss = loss_epoch / data_len if data_len > 0 else 0  # Avoid division by zero
    loss_per_epoch.append(epoch_loss)

    accuracy = accuracy_score(np.array(all_true_labels).flatten(), np.array(all_preds).flatten())
    f1_micro = f1_score(np.array(all_true_labels).flatten(), np.array(all_preds).flatten(), average='micro')
    f1_macro = f1_score(np.array(all_true_labels).flatten(), np.array(all_preds).flatten(), average='macro')
    precision = precision_score(np.array(all_true_labels).flatten(), np.array(all_preds).flatten(), average='macro', zero_division=0)
    recall = recall_score(np.array(all_true_labels).flatten(), np.array(all_preds).flatten(), average='macro', zero_division=0)
    iou = jaccard_index(torch.tensor(all_preds),torch.tensor(all_true_labels), task="multiclass", num_classes=3)

    # Logging
    log_file = "mar28_exp_battery/training_log.txt"
    with open(log_file, "a") as f:
        log_message = (f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, "
                       f"Accuracy: {accuracy:.4f}, F1 Micro: {f1_micro:.4f}, F1 Macro: {f1_macro:.4f}, "
                       f"Precision: {precision:.4f}, Recall: {recall:.4f}, IoU: {iou:.4f}\n"
                       f"Number of patches with no valid pixels: {num_left_count}\n\n")
        # print(log_message, end="")  # Print to console
        f.write(log_message)  # Write to file

    # Save checkpoint
    # Save checkpoint only every 10 epochs
    if epoch == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, accuracy: {accuracy:.4f}, F1 Micro: {f1_micro:.4f}, F1 Macro: {f1_macro:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, IoU: {iou:.4f}")
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, accuracy: {accuracy:.4f}, F1 Micro: {f1_micro:.4f}, F1 Macro: {f1_macro:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, IoU: {iou:.4f}")
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': epoch_loss  # Store averaged loss
        }
        torch.save(checkpoint, f'mar28_exp_battery/checkpoint_epoch_{epoch+1}.pth')
        print(f"Checkpoint saved at epoch {epoch+1}")


# Save losses
import pickle
with open("mar28_exp_battery/loss_per_epoch_100.pkl", "wb") as file:
    pickle.dump(loss_per_epoch, file)
with open("mar28_exp_battery/losses_100.pkl", "wb") as file:
    pickle.dump(losses, file)
with open("mar28_exp_battery/total_true_labels_100.pkl", "wb") as file:
    pickle.dump(total_true_labels, file)
with open("mar28_exp_battery/total_preds_100.pkl", "wb") as file:
    pickle.dump(total_preds, file)

Using device for unet and unet_tiny models : cpu
Using device: cuda:3


/tmp/ipykernel_3001128/2413518608.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  expert_checkpoint = torch.load(unet_path)
/tmp/ipykernel_3001128/2413518608.py:32: Fut

No checkpoint found. Starting from epoch 0.


RuntimeError: Detected more unique values in `target` than expected. Expected only 3 but found 4 in `target`.